### Librerias

In [1]:
import pandas as pd

### Importación de Datos
- Utilizo la base de Ministerio de Salud de la Nación
- Última actualización: 10-09-2020

In [2]:
data = pd.read_csv('Covid19Casos.csv')
print(data.columns)

Index(['id_evento_caso', 'sexo', 'edad', 'edad_años_meses',
       'residencia_pais_nombre', 'residencia_provincia_nombre',
       'residencia_departamento_nombre', 'carga_provincia_nombre',
       'fecha_inicio_sintomas', 'fecha_apertura', 'sepi_apertura',
       'fecha_internacion', 'cuidado_intensivo', 'fecha_cui_intensivo',
       'fallecido', 'fecha_fallecimiento', 'asistencia_respiratoria_mecanica',
       'carga_provincia_id', 'origen_financiamiento', 'clasificacion',
       'clasificacion_resumen', 'residencia_provincia_id', 'fecha_diagnostico',
       'residencia_departamento_id', 'ultima_actualizacion'],
      dtype='object')


#### Tratamiento del Índice

In [3]:
idx = pd.to_datetime(data.fecha_apertura, format='%Y/%m/%d')
data = data.set_index(idx)
data = data.drop('fecha_apertura',axis=1)
data.head(1)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,sepi_apertura,...,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion
fecha_apertura,,,,,,,,,,,,,,,,,,,,,
2020-06-01,1000000,M,53.0,Años,Argentina,CABA,SIN ESPECIFICAR,Buenos Aires,NaN,23,...,NaN,NO,6,Privado,Caso Descartado,Descartado,2,2020-06-09,0,2020-09-10


### Exploración de Datos por provincias

In [4]:
provincias = list(data.carga_provincia_nombre.unique())
provincias.sort(reverse=False)
print(provincias)

['Buenos Aires', 'CABA', 'Catamarca', 'Chaco', 'Chubut', 'Corrientes', 'Córdoba', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tierra del Fuego', 'Tucumán']


#### Cantidad Total de Habitantes que se sometieron a COVID-test

In [5]:
diagnosticos = []

for provincia in provincias:
    df = data.loc[data.carga_provincia_nombre==provincia]
    count = df.sexo.count()
    diagnosticos.append(count)

In [6]:
test = pd.DataFrame(list(zip(provincias, diagnosticos)),columns=['Provincias','Test']).set_index('Provincias')
test.sort_values('Test',ascending=False)

,Test
Provincias,
Buenos Aires,634779
CABA,463953
Córdoba,61925
Santa Fe,53648
Chaco,40250
Mendoza,35281
Jujuy,32403
Tucumán,30451
Río Negro,19710


#### Test Ajustados por 10.000 hab

In [7]:
test_10 = test/10000
test_10

,Test
Provincias,
Buenos Aires,63.4779
CABA,46.3953
Catamarca,0.4093
Chaco,4.0250
Chubut,0.7932
Corrientes,0.8766
Córdoba,6.1925
Entre Ríos,1.4346
Formosa,0.1155


#### Test Ajustados por 100.000 hab

In [8]:
test_100 = test/100000
test_100

,Test
Provincias,
Buenos Aires,6.34779
CABA,4.63953
Catamarca,0.04093
Chaco,0.40250
Chubut,0.07932
Corrientes,0.08766
Córdoba,0.61925
Entre Ríos,0.14346
Formosa,0.01155


#### Test ajustados por población
- Datos de población extraidos de: 
https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/DivisionPolitica

In [9]:
poblacion = [15625084,2890151,367828,1055259,509108,992595,3308876,1235994,530162,673307,
             318951,333642,1738929,1101593,551266,638645,1214441,681055,432310,273964,
             3194537,874006,127205,1448188]

In [10]:
test_ = pd.DataFrame(list(zip(provincias,diagnosticos,poblacion)),
                         columns=['Provincia','Test','Poblacion'])

test_ajus = test_.Test/test_.Poblacion
test_ajus = pd.DataFrame(test_ajus.values,index=test_.Provincia,columns=['Test/Poblacion'])
test_ajus = test_ajus.sort_values('Test/Poblacion',ascending=False)
test_ajus

,Test/Poblacion
Provincia,
CABA,0.160529
Tierra del Fuego,0.061326
Jujuy,0.048125
Buenos Aires,0.040626
Chaco,0.038142
Río Negro,0.030862
La Rioja,0.027895
Santa Cruz,0.024211
Neuquén,0.023074


#### Diagnostico atendido por Instituciones Públicas y Privadas

In [11]:
publico = []
privado = []

for provincia in provincias:
    df = data.loc[data.carga_provincia_nombre==provincia]
    fin_pub = df.loc[df.origen_financiamiento=='Público']
    fin_priv = df.loc[df.origen_financiamiento=='Privado']
    count_pub = fin_pub.sexo.count()
    count_priv = fin_priv.sexo.count()
    publico.append(count_pub)
    privado.append(count_priv)

In [12]:
x = list(zip(provincias,diagnosticos,publico,privado))
res = pd.DataFrame(x,columns=['Provincia','Test','Publico','Privado']).set_index('Provincia')
res.sort_values('Test',ascending=False)

,Test,Publico,Privado
Provincia,,,
Buenos Aires,634779,373787,260991
CABA,463953,212257,251696
Córdoba,61925,38236,23689
Santa Fe,53648,36873,16775
Chaco,40250,37872,2378
Mendoza,35281,20703,14578
Jujuy,32403,25921,6482
Tucumán,30451,28966,1485
Río Negro,19710,17564,2146


In [ ]:
#### Diagnostico discriminado por residencia local y no local

In [14]:
residente = []

for provincia in provincias:
    df = data.loc[data.carga_provincia_nombre==provincia]
    reside = df.loc[df.residencia_provincia_nombre==provincia]
    count = reside.sexo.count()
    residente.append(count)

In [19]:
x = list(zip(provincias,diagnosticos,residente))
res = pd.DataFrame(x,columns=['Provincia','Test','Local']).set_index('Provincia')
res['No_Local'] = res.Test-res.Local
res.sort_values('Test',ascending=False)

,Test,Local,No_Local
Provincia,,,
Buenos Aires,634779,607288,27491
CABA,463953,272661,191292
Córdoba,61925,60964,961
Santa Fe,53648,52533,1115
Chaco,40250,39425,825
Mendoza,35281,34838,443
Jujuy,32403,30499,1904
Tucumán,30451,29063,1388
Río Negro,19710,19358,352
